In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN','NAME'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts().loc[lambda x : x<500].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [4]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = application_df['CLASSIFICATION'].value_counts().loc[lambda x : x<100].index.tolist()
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [5]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dum_df = pd.get_dummies(application_df)
application_dum_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [6]:
# Split our preprocessed data into our features and target arrays
X = application_dum_df.drop(['IS_SUCCESSFUL'], axis=1)
y = application_dum_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [7]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Model 2

In [8]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len( X_train_scaled[0])
hidden_nodes_layer1=80
hidden_nodes_layer2=30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# # Third hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                4000      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 6,461
Trainable params: 6,461
Non-trainable params: 0
_________________________________________________________________


In [9]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [10]:
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5702 - accuracy: 0.7180
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5539 - accuracy: 0.7282
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5513 - accuracy: 0.7290
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5495 - accuracy: 0.7310
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5477 - accuracy: 0.7314
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5463 - accuracy: 0.7317
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5461 - accuracy: 0.7318
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5450 - accuracy: 0.7307
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5441 - accuracy: 0.7333
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5431 - accura

804/804 [==============================] - 1s 2ms/step - loss: 0.5319 - accuracy: 0.7399
Epoch 82/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5323 - accuracy: 0.7400
Epoch 83/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5325 - accuracy: 0.7406
Epoch 84/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5323 - accuracy: 0.7403
Epoch 85/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5322 - accuracy: 0.7396
Epoch 86/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5320 - accuracy: 0.7395
Epoch 87/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5317 - accuracy: 0.7401
Epoch 88/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5316 - accuracy: 0.7404
Epoch 89/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5322 - accuracy: 0.7407
Epoch 90/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5317 - accuracy: 

In [11]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5568 - accuracy: 0.7293 - 564ms/epoch - 2ms/step
Loss: 0.5568006038665771, Accuracy: 0.7293294668197632


# Model 3

In [23]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation="tanh", input_dim=len( X_train_scaled[0])
))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=38, activation="relu"))
nn.add(tf.keras.layers.Dense(units=38, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 50)                2500      
                                                                 
 dense_20 (Dense)            (None, 38)                1938      
                                                                 
 dense_21 (Dense)            (None, 38)                1482      
                                                                 
 dense_22 (Dense)            (None, 1)                 39        
                                                                 
Total params: 5,959
Trainable params: 5,959
Non-trainable params: 0
_________________________________________________________________


In [24]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 945us/step - loss: 0.5662 - accuracy: 0.7222
Epoch 2/100
804/804 [==============================] - 1s 855us/step - loss: 0.5525 - accuracy: 0.7296
Epoch 3/100
804/804 [==============================] - 1s 822us/step - loss: 0.5497 - accuracy: 0.7300
Epoch 4/100
804/804 [==============================] - 1s 795us/step - loss: 0.5481 - accuracy: 0.7324
Epoch 5/100
804/804 [==============================] - 1s 812us/step - loss: 0.5465 - accuracy: 0.7323
Epoch 6/100
804/804 [==============================] - 1s 815us/step - loss: 0.5451 - accuracy: 0.7340
Epoch 7/100
804/804 [==============================] - 1s 817us/step - loss: 0.5452 - accuracy: 0.7332
Epoch 8/100
804/804 [==============================] - 1s 801us/step - loss: 0.5445 - accuracy: 0.7332
Epoch 9/100
804/804 [==============================] - 1s 817us/step - loss: 0.5433 - accuracy: 0.7332
Epoch 10/100
804/804 [==============================] - 1s 806us/step - l